#### 1. 데이터셋 출처
- Pima Indians Diabetes Database|Kaggle
- https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database

#### 1.1 데이터 구성
- Pregnancies: 임신 횟수
- Glucose: 2시간 동안의 경구 포도당 내성 검사에서 혈장 포도당 농도
- BloodPressure:이완기 혈압 (mm Hg)
- SkinThickness: SkinThicknessTriceps 피부 주름 두께(mm)
- Insulin:2시간 혈청 인슐린 (mu U/ml)
- BMI: 체질량 지수(체중kg/ 키(m)^2)
- DiabetesPedigreeFunction: 당뇨병 혈통 기능
- Age: 나이
- Outcome: 768개 중에 268개의 결과 클래스 변수(0 또는 1)는 1이고 나머지는 0입니다.

#### 2 필요한 라이브러리 로드

In [ ]:
# 데이터 분석을 위한 pandas, 수치계산을 위한 numpy
# 시각화를 위한 seaborn, matplotlib.pyplot을 로드합니다.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

#### 3 데이터셋 로드

In [ ]:
df=pd.read_csv("data/diabetes_feature.csv")
df.shape

In [ ]:
# 데이터셋을 미리보기 합니다.

df.head()

In [ ]:
df.columns

In [ ]:
X=df[['Glucose', 'BloodPressure', 'SkinThickness',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Pregnancies_high', 'Insulin_nan',
       'low_glu_insulin']]
X.shape

In [ ]:
y=df['Outcome']
y.shape

#### 4 학습과 예측에 사용할 데이터셋 만들기

In [ ]:
# 사이킷런에서 제공하는 model_selection의 train_test_split으로 만듭니다.
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
# train 세트의 문제와 정답의 데이터 수를 확인해 주세요.

X_train.shape, y_train.shape

In [ ]:
# test 세트의 문제와 정답의 데이터 수를 확인해 주세요.
X_test.shape, y_test.shape

#### 5 학습과 예측하기

In [ ]:
# DecisionTree를 불러옵니다.
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier(max_depth=5,random_state=42)
model

In [ ]:
# 학습을 시킵니다.
model.fit(X_train,y_train)

In [ ]:
# 최적의 max_depth 찾기
for max_depth in range(3,12):
  model=DecisionTreeClassifier(max_depth=max_depth,random_state=42)
  y_pred=model.fit(X_train,y_train).predict(X_test)
  score=accuracy_score(y_test,y_pred)*100
  print(max_depth,score)

#### 6 Grid Search
- 모델의 하이퍼파라미터에 넣을 수 있는 값들을 순차적으로 입력한 뒤에 가장 높은 성능을 보이는 하이퍼파라미터를 찾는 탐색 방법

In [ ]:
from sklearn.model_selection import GridSearchCV

model=DecisionTreeClassifier(random_state=42)
# param_grid: 튜닝하고 싶은 파라미터 정보를 넣는다.
# max_features: 일부 feature만 사용하고 싶을 때 사용. 1은 전체를 의미
param_grid={"max_depth":range(3,12),"max_features":[0.3,0.5,0.7,0.9,1.0]}
# n_jobs=-1로 설정하여 사용 가능한 모든 프로세스를를 학습에 이용한다.
# verbose: 1로 설정하여 로그를 찍으면서 학습한다. 0이면 로그 출력 X
clf=GridSearchCV(model,param_grid=param_grid,n_jobs=-1, cv=5,verbose=2)
clf.fit(X_train,y_train)

In [ ]:
clf.best_params_

In [ ]:
clf.best_estimator_

In [ ]:
clf.best_score_

In [ ]:
pd.DataFrame(clf.cv_results_).sort_values(by="rank_test_score").head() # cross validation 결과 반환

In [ ]:
clf.predict(X_test)

In [ ]:
clf.score(X_test,y_test)

In [ ]:
feature_names=X_train.columns.tolist()

In [ ]:
from sklearn.tree import plot_tree
plt.figure(figsize=(15,15))
tree=plot_tree(model,feature_names=feature_names,fontsize=10,filled=True)

In [ ]:
# 예측을 하고 결과를 y_predict에 담습니다.
y_pred=model.predict(X_test)
y_pred

#### 7 정확도(Accuracy) 측정하기

In [ ]:
# 다르게 예측한 갯수를 구해서 diff_count에 할당해 줍니다.
# 정답과 예측값이 다른 경우 1 or -1로 나옴
abs(y_pred - y_test).sum()

In [ ]:
# accuracy score를 구합니다.
from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_pred)*100

In [ ]:
max_depth=np.random.randint(3,20,10)
max_depth

In [ ]:
max_features=np.random.randint(0.7,1.0,100)
max_features

#### 8 RandomSearchCV

In [ ]:
param_distributions={"max_depth":max_depth,"max_features":max_features,"min_samples_split":list(range(2,7))}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

clf = RandomizedSearchCV(model,
    param_distributions,
    n_iter=1000,
    scoring="accuracy",
    n_jobs=-1,
    cv=5,
    random_state=42
    )

clf.fit(X_train, y_train)

In [ ]:
clf.best_params_

In [ ]:
clf.best_score_

In [ ]:
clf.score(X_test,y_test)

In [ ]:
pd.DataFrame(clf.cv_results_).sort_values(by="rank_test_score").head()